# Générateur de texte

In [1]:
!pip install numpy
!pip install scipy
!pip install matplotlib

import re
import glob
import numpy as np
import scipy
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [8.6, 6]
#plt.rcParams['figure.dpi'] = 100

## Exemple de données d'apprentissage

In [2]:
import re
with open('data/swan7.txt') as f:
    line = f.readline()[:-1] # -1 pour supprimer la fin de ligne
print(f'Avant traitement: {line}')
line = re.sub(r'[^a-z]+', ' ', line.lower())
print(f'Après traitement: {line}')

Avant traitement: Longtemps, je me suis couche de bonne heure. Parfois, a peine ma
Après traitement: longtemps je me suis couche de bonne heure parfois a peine ma


## Tokenization / embedding

In [3]:
alphabet = 'abcdefghijklmnopqrstuvwxyz '
ntoks = len(alphabet)
num2alpha = dict(enumerate(alphabet))
alpha2num= {v:k for k,v in num2alpha.items()}
print(f'Nombre de token: {ntoks}')
print(f'alphabet -> nombre: {alpha2num}')
print(f'nombre -> alphabet: {num2alpha}')


Nombre de token: 27
alphabet -> nombre: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, ' ': 26}
nombre -> alphabet: {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z', 26: ' '}


In [4]:
tokens = np.fromiter((alpha2num[x] for x in line), dtype=int)
print(f'tokens: {tokens[:20]}')

tokens: [11 14 13  6 19  4 12 15 18 26  9  4 26 12  4 26 18 20  8 18]


## Apprentissage

In [5]:
ordre = 3
table = np.ones([ntoks] * ordre, dtype=int)

In [6]:
for file in glob.iglob('data/*.txt'):
    with open(file) as f:
        prefix = np.zeros(0, dtype=int)
        for line in f:
            line = re.sub(r'[^a-z]+', ' ', line.lower())
            tokens = np.concat((prefix, np.fromiter((alpha2num[x] for x in line), dtype=int)))
            for i in range(tokens.size - ordre):
                table[tuple(tokens[i:i+ordre])] += 1
            prefix = tokens[-ordre:]
ltable = np.log(1 + table)

In [7]:
print(f'Nombre de tuples: {table.sum()}')
print(f'Nombre d\'entrées non vides: {np.sum(table > 1)}')

Nombre de tuples: 2285240
Nombre d'entrées non vides: 4363


In [8]:
if ordre == 2:
    fig, ax = plt.subplots()
    ax.set_xticks(range(ntoks),labels=alphabet)
    ax.set_yticks(range(ntoks), labels=alphabet)
    ax.matshow(table)

## Modèles génératifs et géneration de texte

In [9]:
from scipy.special import softmax
temperature = 1.2
longueur = 120

tokens = [alpha2num[c] for c in 'il etait une fois ']
for i in range(longueur):
    idx = tuple(tokens[1-ordre:])
    new_tok = np.random.choice(ntoks, p=softmax(ltable[idx]/1.2))
    tokens.append(new_tok)
texte = ''.join(num2alpha[t] for t in tokens)
print(texte)

il etait une fois une elarqui a vous l on de pon et j y aves se ra soit il  es on affales licen voisses efoils comes te taitel diustet que
